In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [7]:
df_config = pd.read_csv('sup_pri_config_20230517.csv')
df_config.head()

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score
0,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0,1000,last_mile_distance,0,100000,1
1,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000,2000,last_mile_distance,0,100000,1
2,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,2.0,3.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000,3000,last_mile_distance,0,100000,1
3,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,3.0,4.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,3000,4000,last_mile_distance,0,100000,1
4,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,afternoon,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0,1000,last_mile_distance,0,100000,1


In [9]:
df_config.columns

Index(['service_detail_id', 'hex_id', 'day_name', 'time_period',
       'bucket_start', 'bucket_end', 'bucket_type',
       'captain_segment_less_riders', 'deprioritization',
       'drop_location_preference', 'earnings_per_hour', 'eta',
       'first_mile_distance', 'handholding', 'micro_corridor',
       'probability_of_accept', 'route_preference',
       'dropped_to_total_pings_ratio', 'service_affinity',
       'high_last_mile_distance_affinity', 'prioritization_stage',
       'captain_segment', 'primary_bucket_type', 'primary_bucket_start',
       'primary_bucket_end', 'secondary_bucket_type', 'secondary_bucket_start',
       'secondary_bucket_end', 'config_priority_score'],
      dtype='object')

In [10]:
columns = ['captain_segment_less_riders', 'deprioritization',
       'drop_location_preference', 'earnings_per_hour', 'eta',
       'first_mile_distance', 'handholding', 'micro_corridor',
       'probability_of_accept', 'route_preference',
       'dropped_to_total_pings_ratio', 'service_affinity',
       'high_last_mile_distance_affinity', 'primary_bucket_start',
       'primary_bucket_end',  'secondary_bucket_start',
       'secondary_bucket_end', 'config_priority_score']
for column in columns:
    df_config[column] = df_config[column].astype(float)
df_config.dtypes

service_detail_id                    object
hex_id                               object
day_name                             object
time_period                          object
bucket_start                        float64
bucket_end                          float64
bucket_type                          object
captain_segment_less_riders         float64
deprioritization                    float64
drop_location_preference            float64
earnings_per_hour                   float64
eta                                 float64
first_mile_distance                 float64
handholding                         float64
micro_corridor                      float64
probability_of_accept               float64
route_preference                    float64
dropped_to_total_pings_ratio        float64
service_affinity                    float64
high_last_mile_distance_affinity    float64
prioritization_stage                 object
captain_segment                     float64
primary_bucket_type             

In [12]:
df_config['configs_to_remove'] = np.where(((df_config['hex_id'].isin(hexes)) & (df_config['time_period'] == 'morning_peak') & 
                                           (df_config['service_detail_id'] == '5da4660028af187d8d52cc3c') & (df_config['day_name'] == 'all')), 'remove', 'not_remove')
df_config[df_config['configs_to_remove'] == 'remove'] 

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score,configs_to_remove
248062,5da4660028af187d8d52cc3c,883da111abfffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248072,5da4660028af187d8d52cc3c,883da11ad7fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248074,5da4660028af187d8d52cc3c,883da11a99fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248075,5da4660028af187d8d52cc3c,883da111a5fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248077,5da4660028af187d8d52cc3c,883da11a9bfffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248078,5da4660028af187d8d52cc3c,883da111a1fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248080,5da4660028af187d8d52cc3c,883da1118dfffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248082,5da4660028af187d8d52cc3c,883da11185fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove
248083,5da4660028af187d8d52cc3c,883da111a9fffff,all,morning_peak,0.0,20000.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,20000000.0,last_mile_distance,0.0,100000.0,1.0,remove


In [13]:
df_config_filter = df_config[df_config['configs_to_remove'] != 'remove']
df_config_filter

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score,configs_to_remove
0,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
1,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
2,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,2.0,3.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,3000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
3,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,3.0,4.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,3000.0,4000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
4,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,afternoon,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606851,5da4660028af187d8d52cc3c,883da111a5fffff,Saturday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,40.0,0.0,-60.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
606852,5da4660028af187d8d52cc3c,883da111a5fffff,Saturday,morning_peak,2.0,100.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,100000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
606853,5da4660028af187d8d52cc3c,883da111a5fffff,Sunday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,5.0,0.0,-95.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
606854,5da4660028af187d8d52cc3c,883da111a5fffff,Sunday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,40.0,0.0,-60.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,0.0,not_remove


In [15]:
df_configs_to_add = df_config_filter[(df_config_filter['hex_id'].isin(hexes)) & (df_config_filter['time_period'] == 'morning_peak') &
                                      (df_config['service_detail_id'] == '5da4660028af187d8d52cc3c')]
df_configs_to_add.head()

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score,configs_to_remove
605218,5da4660028af187d8d52cc3c,883da11a83fffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,5.0,0.0,-95.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
605219,5da4660028af187d8d52cc3c,883da11a83fffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,40.0,0.0,-60.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
605220,5da4660028af187d8d52cc3c,883da11a83fffff,Monday,morning_peak,2.0,100.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,100000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
605221,5da4660028af187d8d52cc3c,883da11a83fffff,Tuesday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,5.0,0.0,-95.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,0.0,not_remove
605222,5da4660028af187d8d52cc3c,883da11a83fffff,Tuesday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,40.0,0.0,-60.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,0.0,not_remove


In [18]:
df_configs_to_add['time_period'] = 'afternoon'
df_configs_to_add2 = df_configs_to_add.copy()
df_configs_to_add2['time_period'] = 'evening_peak'
df_final = pd.concat([df_config_filter, df_configs_to_add, df_configs_to_add2])
df_final.head()

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score,configs_to_remove
0,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
1,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
2,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,2.0,3.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,3000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
3,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,3.0,4.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,3000.0,4000.0,last_mile_distance,0.0,100000.0,1.0,not_remove
4,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,afternoon,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0,not_remove


In [19]:
df_final = df_final.drop(['configs_to_remove'], axis = 1)
df_final.head()

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score
0,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0
1,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,1.0
2,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,2.0,3.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,3000.0,last_mile_distance,0.0,100000.0,1.0
3,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,3.0,4.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,3000.0,4000.0,last_mile_distance,0.0,100000.0,1.0
4,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,afternoon,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0


In [20]:
df_final.to_parquet('sup_pri_configs_staging_2030517.parquet', index = False)

In [21]:
new_configs = pd.read_parquet('sup_pri_configs_staging_2030517.parquet')
new_configs.head()

,service_detail_id,hex_id,day_name,time_period,bucket_start,bucket_end,bucket_type,captain_segment_less_riders,deprioritization,drop_location_preference,earnings_per_hour,eta,first_mile_distance,handholding,micro_corridor,probability_of_accept,route_preference,dropped_to_total_pings_ratio,service_affinity,high_last_mile_distance_affinity,prioritization_stage,captain_segment,primary_bucket_type,primary_bucket_start,primary_bucket_end,secondary_bucket_type,secondary_bucket_start,secondary_bucket_end,config_priority_score
0,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0
1,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,1.0,2.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,1000.0,2000.0,last_mile_distance,0.0,100000.0,1.0
2,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,2.0,3.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,2000.0,3000.0,last_mile_distance,0.0,100000.0,1.0
3,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,morning_peak,3.0,4.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,3000.0,4000.0,last_mile_distance,0.0,100000.0,1.0
4,5c7456d827dd21496ab1d2b7,883d13915dfffff,Monday,afternoon,0.0,1.0,first_mile_distance,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dispatch,0.0,first_mile_distance,0.0,1000.0,last_mile_distance,0.0,100000.0,1.0


In [26]:
new_configs['rows_to_remove'] = np.where(((new_configs['hex_id'].isin(hexes)) & (new_configs['time_period'] == 'evening_peak') & 
                                 (new_configs['service_detail_id'] == '5da4660028af187d8d52cc3c') & (new_configs['day_name'] == 'all')), 'remove', 'not_remove')

In [27]:
new_configs_filter = new_configs[new_configs['rows_to_remove'] == 'not_remove']
new_configs_filter.shape

(609161, 30)

In [29]:
new_configs_filter = new_configs_filter.drop(['rows_to_remove'], axis = 1)

In [30]:
new_configs_filter.to_parquet('sup_pri_configs_staging_2030517v2.parquet', index = False)

In [11]:
df = pd.read_csv('mc_hexes.csv')
hexes = df['hex_id'].unique().tolist()
print(hexes)

['883da11a83fffff', '883da11ab1fffff', '883da118cbfffff', '883da118c9fffff', '883da11ab5fffff', '883da11aabfffff', '883da11abbfffff', '883da11abdfffff', '883da1188bfffff', '883da11aa1fffff', '883da11aa3fffff', '883da11aa7fffff', '883da11aa5fffff', '883da118cdfffff', '883da118c1fffff', '883da118c3fffff', '883da11ab7fffff', '883da1181bfffff', '883da118d7fffff', '883da118c7fffff', '883da11a87fffff', '883da118ddfffff', '883da118d3fffff', '883da118c5fffff', '883da11ab9fffff', '883da11889fffff', '883da11a97fffff', '883da118d1fffff', '883da11a95fffff', '883da11ab3fffff', '883da118d9fffff', '883da118d5fffff', '883da1036dfffff', '883da10369fffff', '883da11aadfffff', '883da1188dfffff', '883da118e9fffff', '883da118dbfffff', '883da10365fffff', '883da11a85fffff', '883da1189dfffff', '883da11aa9fffff', '883da11813fffff', '883da11881fffff', '883da118ebfffff', '883da11a81fffff', '883da118e3fffff', '883da10325fffff', '883da1185bfffff', '883da11a99fffff', '883da11a91fffff', '883da111a1fffff', '883da11a9b

In [6]:
#Query data from order_logs_snapshot

q = """
select * from iDispatch.supply_prioritizer_strategy_config_vw
"""


#Load data into pandas table
df_copy2 = pd.read_sql(q, connection)

DatabaseError: {'message': "line 2:15: Schema 'idispatch' does not exist", 'errorCode': 45, 'errorName': 'SCHEMA_NOT_FOUND', 'errorType': 'USER_ERROR', 'errorLocation': {'lineNumber': 2, 'columnNumber': 15}, 'failureInfo': {'type': 'io.trino.spi.TrinoException', 'message': "line 2:15: Schema 'idispatch' does not exist", 'suppressed': [], 'stack': ['io.trino.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:48)', 'io.trino.sql.analyzer.SemanticExceptions.semanticException(SemanticExceptions.java:43)', 'io.trino.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:1789)', 'io.trino.sql.analyzer.StatementAnalyzer$Visitor.visitTable(StatementAnalyzer.java:445)', 'io.trino.sql.tree.Table.accept(Table.java:60)', 'io.trino.sql.tree.AstVisitor.process(AstVisitor.java:27)', 'io.trino.sql.analyzer.StatementAnalyzer$Visitor.process(StatementAnalyzer.java:462)', 'io.trino.sql.analyzer.StatementAnalyzer$Visitor.analyzeFrom(StatementAnalyzer.java:3668)', '

In [224]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8
from orders.order_logs_snapshot
where service_obj_service_name = 'Link'
and order_id = '6464617599b931616ed29279'
"""


#Load data into pandas table
df_copy2 = pd.read_sql(q, connection)

In [225]:
df_copy2

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8


In [206]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, event_type, drop_location_hex_8
from orders.order_logs_immutable
where yyyymmdd = '20230516'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and pickup_location_hex_8 in ('883da11aa3fffff', '883da11ab5fffff', '883da11abdfffff')
and drop_location_hex_8 in ('883da11a83fffff', '883da11ab1fffff', '883da118cbfffff', '883da118c9fffff', '883da11ab5fffff', '883da11aabfffff', '883da11abbfffff',
 '883da11abdfffff', '883da1188bfffff', '883da11aa1fffff', '883da11aa3fffff', '883da11aa7fffff', '883da11aa5fffff', '883da118cdfffff', '883da118c1fffff',
   '883da118c3fffff', '883da11ab7fffff', '883da1181bfffff', '883da118d7fffff', '883da118c7fffff', '883da11a87fffff', '883da118ddfffff', '883da118d3fffff',
     '883da118c5fffff', '883da11ab9fffff', '883da11889fffff', '883da11a97fffff', '883da118d1fffff', '883da11a95fffff', '883da11ab3fffff', '883da118d9fffff',
       '883da118d5fffff', '883da1036dfffff', '883da10369fffff', '883da11aadfffff', '883da1188dfffff', '883da118e9fffff', '883da118dbfffff', '883da10365fffff',
         '883da11a85fffff', '883da1189dfffff', '883da11aa9fffff', '883da11813fffff', '883da11881fffff', '883da118ebfffff', '883da11a81fffff', '883da118e3fffff',
           '883da10325fffff', '883da1185bfffff', '883da11a99fffff', '883da11a91fffff', '883da111a1fffff', '883da11a9bfffff', '883da111a9fffff', '883da11a8dfffff',
             '883da11185fffff', '883da111abfffff', '883da1118dfffff', '883da11a89fffff', '883da11883fffff', '883da1180bfffff', '883da11a9dfffff', '883da118e7fffff',
               '883da11ae3fffff', '883da11815fffff', '883da11851fffff', '883da11ad7fffff', '883da118abfffff', '883da11853fffff', '883da11857fffff', '883da11819fffff',
                 '883da1181dfffff', '883da11811fffff', '883da11817fffff', '883da11885fffff', '883da11ae7fffff', '883da11a8bfffff', '883da111a5fffff')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [207]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,event_type,drop_location_hex_8
0,646322c407d1771134f4ab5f,NaN,Delhi,883da11abdfffff,28.448126,77.071922,115924,11,1145,Link,11:59,2,20230516,new,None,3.0,"[""628d274d991337220f35a1d6"",""63e0beb93ee97b83a...",883da11abdfffff,62ffad0b33f1010360fd37f1,rider_busy,883da11a95fffff
1,646321819d1dd7505bb8abe6,NaN,Delhi,883da11abdfffff,28.449860,77.071190,115401,11,1145,Link,11:59,2,20230516,new,None,3.0,"[""62456120a060e81479ee5b85"",""636a2ea1748286f1c...",883da11abdfffff,6327b441e21bd94ab3cbf89c,rider_reject,883da11a95fffff
2,646321f571207e33351cfb70,NaN,Delhi,883da11ab5fffff,28.443150,77.080490,115557,11,1145,Link,11:59,2,20230516,onTheWay,None,2.0,"[""61d14357a75746a0d2e577a4"",""6421d0e7c541daa6c...",883da11ab5fffff,61d14357a75746a0d2e577a4,accepted,883da11ae3fffff
3,6462f5b3a5c51b1fbd43b395,NaN,Delhi,883da11ab5fffff,28.438669,77.082596,084707,08,0845,Link,08:59,2,20230516,new,None,5.0,"[""5e0ee696715b3f22b963eea8"",""5e1ad3a72f1e6d5f5...",883da11ab5fffff,5e1ad3a72f1e6d5f599bc0a1,rider_busy,883da118d1fffff
4,6463026f9c54a2545608513b,NaN,Delhi,883da11aa3fffff,28.439182,77.074005,094127,09,0930,Link,09:59,2,20230516,new,None,4.0,"[""5ca33ecc38f05017d5788873"",""605f2f042c4cf805e...",883da11aa3fffff,605f2f042c4cf805e7185c29,rider_busy,883da11abdfffff


In [208]:
df['order_id'].nunique()

385

In [209]:
df['hour'].unique()

array(['11', '08', '09', '10'], dtype=object)

In [210]:
df['map_riders'] = df['map_riders'].apply(lambda x: json.loads(x))

In [211]:
df['mc_captain_1'] = df['map_riders'].apply(lambda x: '59b153ef0c3e1ca306385c51' in x)
df['mc_captain_2'] = df['map_riders'].apply(lambda x: '5daff8e0b2d9ba1c1cf1bdf9' in x)

In [213]:
df[df['mc_captain_1'] == True]

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,event_type,drop_location_hex_8,mc_captain_1,mc_captain_2


In [180]:
df[df['order_id'] == '64630d83ec83aa684aa3642f']

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,event_type,mc_captain_1,mc_captain_2
76514,64630d83ec83aa684aa3642f,NaN,Delhi,883da11abbfffff,28.454397,77.086586,102843,10,1015,Link,10:00,2,20230516,onTheWay,None,2.0,"[633d1b1840ea6e20511ca631, 638b1b402ba217e90f0...",883da11abbfffff,633d1b1840ea6e20511ca631,accepted,True,False


In [164]:
first_captain_orders = df[df['mc_captain_1'] == True]['order_id'].unique().tolist()
second_captain_orders = df[df['mc_captain_2'] == True]['order_id'].unique().tolist()

In [165]:
print(first_captain_orders)

['64630bea396d452aefce0960', '6462fd8311a4ec484f00d856', '646315f39c54a25456087b18', '646314ee4bcb4717d771cf66', '6462fd1dab756b601aec27af', '6462fecd9d1dd7505bb86585', '6463040e850c6f6b83000c92', '6462ffa689d713204bc1a67b', '6462fda2f132af47ee75abe1', '6462fe50ab756b601aec2a78', '6463017f8b31b21a90c69746', '64630322e5282a5d706eb2d3', '6462fc464e73755d6de51819', '6463090211a4ec484f00f35f', '646300a64bcb4717d771a319', '6462fe53ab756b601aec2a81', '6463054af132af47ee75bd9d', '6462fceeec83aa684aa33fd9', '64630816f132af47ee75c426', '6462fd134bcb4717d7719afc', '646302f8e5282a5d706eb265', '6462fa748b31b21a90c6866f', '64630a5671207e33351ccefd', '64630d75602d9a5dcb0ed1d5', '6463047da76666026f44f7c4', '6462fd1b11a4ec484f00d762', '6462fc0ea76666026f44e30a', '6463023f07d1771134f46a75', '646301578e0c1952b3889285', '6462f933a76666026f44dc39', '6462ff2e9d1dd7505bb86661', '6462fa51a76666026f44dec6', '64630e1d4e73755d6de54145', '646308e707d1771134f479d2', '6463055a2463953d1030184b', '646300b7850c6f6b83

In [168]:
print(len(first_captain_orders))

131


In [167]:
print(second_captain_orders)

[]


In [169]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230516'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id in ('64630bea396d452aefce0960', '6462fd8311a4ec484f00d856', '646315f39c54a25456087b18', '646314ee4bcb4717d771cf66', '6462fd1dab756b601aec27af',
 '6462fecd9d1dd7505bb86585', '6463040e850c6f6b83000c92', '6462ffa689d713204bc1a67b', '6462fda2f132af47ee75abe1', '6462fe50ab756b601aec2a78',
   '6463017f8b31b21a90c69746', '64630322e5282a5d706eb2d3', '6462fc464e73755d6de51819', '6463090211a4ec484f00f35f', '646300a64bcb4717d771a319',
     '6462fe53ab756b601aec2a81', '6463054af132af47ee75bd9d', '6462fceeec83aa684aa33fd9', '64630816f132af47ee75c426', '6462fd134bcb4717d7719afc',
       '646302f8e5282a5d706eb265', '6462fa748b31b21a90c6866f', '64630a5671207e33351ccefd', '64630d75602d9a5dcb0ed1d5', '6463047da76666026f44f7c4',
         '6462fd1b11a4ec484f00d762', '6462fc0ea76666026f44e30a', '6463023f07d1771134f46a75', '646301578e0c1952b3889285', '6462f933a76666026f44dc39',
           '6462ff2e9d1dd7505bb86661', '6462fa51a76666026f44dec6', '64630e1d4e73755d6de54145', '646308e707d1771134f479d2', '6463055a2463953d1030184b',
             '646300b7850c6f6b83000483', '6462fd2da76666026f44e62a', '646309c9ab756b601aec44e3', '646307c871207e33351cc94c', '646302e0ec83aa684aa34cf7',
               '646304fb07d1771134f470fc', '64630b3cec83aa684aa35f6c', '64630607850c6f6b8300114b', '64631601f132af47ee75e10f', '646304018b31b21a90c69d83',
                 '6463067b396d452aefcdfd68', '6463022b602d9a5dcb0eb8a5', '6462fd3aab756b601aec27f2', '6462fb8d9d1dd7505bb85d6e', '6462fb0989d713204bc19b7f',
                   '646308fe11a4ec484f00f356', '646301c1850c6f6b8300070a', '646301ad8b31b21a90c697a2', '64630b668e0c1952b388a97c', '6462fdb28b31b21a90c68e70',
                     '64630bdc4e73755d6de53c75', '64630197a5c51b1fbd43d008', '646306252463953d10301a32', '6462fcea520fd368ce490d0f', '6462ff179c54a25456084961',
                       '6462f8fd850c6f6b83fff238', '6462fceaa5c51b1fbd43c4f3', '646308fc9d1dd7505bb87d6e', '6462fe20850c6f6b83fffeb6', '6463092289d713204bc1bcc0',
                         '646306f3850c6f6b83001356', '64630a5689d713204bc1bf52', '6462fe909ad8e2253f177dda', '64630d83ec83aa684aa3642f', '6462fea371207e33351cb3b9',
                           '646304bba5c51b1fbd43d735', '6462fea5ab756b601aec2b47', '64630504520fd368ce49203a', '64630176c010e5281fa4c4f6', '64630246e5282a5d706eb0d8',
                             '64630a304e73755d6de538f3', '646302a79ad8e2253f17867a', '64630c98995a513dd183a52b', '646306d4396d452aefcdfe51',
                               '64630733f132af47ee75c221', '6463016e520fd368ce49174f', '64630d739ad8e2253f179eae', '64630177396d452aefcdf19d',
                                 '6462fb5b9da2f91768944423', '64631596ab756b601aec5d41', '646307a49da2f917689461ce', '6462fbb75573745a331fa0ec',
                                   '6462fe1311a4ec484f00d9a5', '6462fdf3520fd368ce490f77', '646301f4ec83aa684aa34ad1', '64630a029ad8e2253f179777',
                                     '6462ff4d8e0c1952b3888e32', '6463066fec83aa684aa35521', '6462f9c2ab756b601aec1f41', '6462fa7d2463953d102ffe2f',
                                       '646303df89d713204bc1b051', '646304cf520fd368ce491fb0', '6462f9404e73755d6de5109d', '646300f69ad8e2253f17829c',
                                         '64630eef4e73755d6de5431f', '64630e87396d452aefce0ec5', '6462ff3289d713204bc1a57c', '6462fb05a76666026f44e06c',
                                           '646301a4c010e5281fa4c564', '646301f74e73755d6de525bd', '646304708e0c1952b38899c4', '64630bc0f132af47ee75cc4c',
                                             '6462fd2b850c6f6b83fffc56', '6463047107d1771134f46fca', '6462f9cbec83aa684aa3383d', '6462fa6d8e0c1952b388824b',
                                               '64630aa0a5c51b1fbd43e4a1', '6462fddf5573745a331fa6b0', '64630e33520fd368ce493419', '6463052f71207e33351cc32a',
                                                 '646300f5396d452aefcdf06a', '646310845573745a331fcfed', '64630865ab756b601aec41ca', '6463020a07d1771134f46a01',
                                                   '64630604850c6f6b83001144', '6463069d89d713204bc1b6ff', '6462fd42602d9a5dcb0ead78', '6462fc47ab756b601aec2566',
                                                     '646304f59ad8e2253f178c29', '64630ed48e0c1952b388b075', '6462ff932463953d10300a55', '646306c5a5c51b1fbd43dbf2',
                                                       '64630c5d4e73755d6de53d7b', '6462ff2d9c54a25456084999', '6462fdff9da2f91768944acb', '6462ff3a89d713204bc1a593')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [171]:
df_dispatch_copy2['captains_with_score'] = df_dispatch_copy2['captains_with_score'].apply(lambda x: json.loads(x))

In [178]:
df_dispatch_copy2['order_id'].nunique()

125

In [177]:
df_dispatch_copy2['enabled_strategies_with_config'].values[0]

'{"first_mile_distance":{"bucket":{"params":[{"primaryBucketEnd":1000,"primaryBucketStart":0,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":5},{"primaryBucketEnd":2000,"primaryBucketStart":1000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":40},{"primaryBucketEnd":100000,"primaryBucketStart":2000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100}],"type":"first_mile_distance"}}}'

In [185]:
df_dispatch_copy2.loc[0, :]

created_on                                                            1684213124680
event_id                                       fc453ad0-302a-428a-3c52-0d965c65ab93
updated_at                                                            1684213124680
version                                                                       1.3.0
enabled_captain_filter_types      ["economicFeasibilityOfARide","failedMatchInTh...
order_id                                                   64630d83ec83aa684aa3642f
compositeHexId                                                                     
prioritization_stage                                                       dispatch
enabled_strategies_with_config    {"first_mile_distance":{"bucket":{"params":[{"...
filtered_captains                                                                []
captains                          [{"earnings":{"amount":58.26,"preCommissionAmo...
city_id                                                    5bc5ac2312477c2ec

In [200]:
fm_distance = []
captain_rank = []
micro_corridor = []
location = []
captain_list = df_dispatch_copy2['captains_with_score'].values
for captains in captain_list:
    for captain in captains:
        if captain['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance.append(captain['first_mile_distance'])
            captain_rank.append(captains.index(captain))
            micro_corridor.append(captain['score']['micro_corridor'])
            location.append(h3.geo_to_h3(captain['location']['lat'], captain['location']['lng'], resolution=8))
            

In [201]:
print(fm_distance)

[2671.3, 2078.9, 2836.2, 2117.7, 2855.9, 2225.5, 2917.1, 2276.1, 3614.8, 2622.9, 2241.2, 2361.0, 3025.0, 2632.4, 2772.5, 2772.5, 2721.7, 2449.3, 2245.7, 2288.6, 2448.3, 2218.9, 2155.1, 2650.9, 2316.2, 2574.7, 2633.6, 2366.3, 2263.3, 3829.1, 2446.5, 2112.9, 2868.4, 2418.4, 2607.7, 3035.6, 2775.7, 2128.3, 2277.2, 2528.7, 3519.0, 3750.5, 2301.7, 2163.0, 2786.9, 2357.1, -1.0, 2679.8, 2269.4, 3707.1, 2369.0, 2361.5, 2866.4, 2229.1, 2055.2, 2405.7, 2303.9, 3816.7, 3379.7, 2423.3, 2999.9, 2208.1, -1.0, 2398.3, 2667.8, 2315.2, 2990.9, 2744.8, 2312.4, 2469.0, 2273.0, 3595.7, 2304.5, 2201.8, 3780.7, 2216.3, 2185.3, 2294.7, 2353.2, 2796.6, 2864.2, 2330.4, 2378.5, 3780.6, 2380.3, 2250.4, 2444.4, 2355.8, 2154.0, 2076.4, 2447.0, 2348.7, 3110.4, 2361.6, 2257.9, 2112.0, 2526.6, 2225.1, 2182.1, 3601.3, 2167.4, 2420.4, 2309.6, 2154.5, 2795.3, 2377.5, 3591.0, 2563.0, 2696.2, 2399.7, 2359.2, 2202.1, 2230.0, 2293.0, 2748.2, 2548.7, 3412.2, 2256.7, 2269.3, 2814.4, 2194.3, 3037.6, 2249.8, 2731.1, 2311.6, 272

In [205]:
set(location)

{'883da11aa3fffff', '883da11ab5fffff', '883da11abdfffff'}

In [174]:
print(captain_rank)

[43, 19, 27, 7, 41, 20, 36, 4, 26, 24, 12, 11, 36, 61, 24, 21, 36, 54, 19, 2, 57, 19, 6, 97, 19, 37, 24, 12, 14, 63, 6, 6, 7, 7, 84, 81, 16, 4, 8, 11, 50, 74, 4, 5, 76, 46, 6, 43, 10, 63, 9, 12, 7, 6, 35, 5, 8, 12, 61, 14, 9, 3, 22, 5, 3, 3, 12, 3, 5, 16, 4, 22, 7, 1, 12, 6, 3, 7, 11, 5, 12, 7, 4, 11, 4, 1, 4, 6, 2, 3, 2, 4, 127, 3, 8, 6, 10, 3, 3, 17, 3, 2, 1, 7, 7, 5, 19, 45, 43, 10, 13, 10, 6, 5, 10, 14, 72, 76, 3, 4, 5, 10, 7, 3, 17, 10, 6, 6, 4, 6, 3, 12, 60, 20, 20, 39, 9, 4, 4, 13]


In [175]:
print(micro_corridor)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [42]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230515'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id in ('6461dd89780955423c323d29', '6461d9b297b0a30bbd66f587', '6461dccd8753985ab62d977e', '6461dbeb8753985ab62d9584', '6461db90b97fe77d4ecf91b7',
 '6461d74e751ce46ce048ea48', '6461d7141e1f7e2e37653760', '6461dae5ca5873133f31596f', '6461d9938753985ab62d9080', '6461d8cf8956b6528798490c',
   '6461dccb780955423c323b80', '6461d9385ac5d8625a819c48', '6461dae5cd5d212b73a0eb1e', '6461da5c9c50b541b2df2630', '6461dcf643583753739896d8',
     '6461df76751ce46ce048fbb9', '6461d5bd5ac5d8625a819502', '6461d754780955423c322f8d', '6461ddcc8eac592e5cae9d4e', '6461dee44358375373989aed',
       '6461d823780955423c32315d', '6461da858eac592e5cae964c', '6461dad19f166a55ff604990', '6461dd0c9f166a55ff604e47', '6461dee4b97fe77d4ecf98c6',
         '6461d72d8753985ab62d8b4b', '6461d6ff43583753739889d9', '6461dbcfa78efb0c7dc72b91', '6461dc088eac592e5cae996a', '6461d64db97fe77d4ecf861b',
           '6461da69ca5873133f31585d', '6461d6b61e1f7e2e3765368a', '6461df4bf4a8ae312cd1f650', '6461dd2fa78efb0c7dc72e9a', '6461d8d39c50b541b2df22f7',
             '6461dbc09f166a55ff604ba6', '6461dd46cd5d212b73a0f01d', '6461dcaf8eac592e5cae9ab3', '6461db618eac592e5cae9809', '6461dbb91e1f7e2e37654165',
               '6461de1df4a8ae312cd1f39c', '6461d94f780955423c3233cd', '6461de08cd5d212b73a0f1cc', '6461dce9ad68a90a314dce9f', '6461de36751ce46ce048f8fe',
                 '6461d5fecd5d212b73a0e034', '6461d79eb97fe77d4ecf8910', '6461df5597b0a30bbd6701d5', '6461dde51e1f7e2e3765465f', '6461de5a5ac5d8625a81a759',
                   '6461d9dc1e1f7e2e37653d86', '6461d6f643583753739889c8', '6461d60c1e1f7e2e3765351d', '6461dc8943583753739895fc', '6461dedf9c50b541b2df2fc2',
                     '6461d59cf4a8ae312cd1e0f8', '6461da7a9c50b541b2df266b', '6461d5a1a78efb0c7dc71e6e', '6461d75d8753985ab62d8bb2', '6461d7ff612b71236703b50f',
                       '6461d9b9780955423c3234d6', '6461dc9b612b71236703bf5b', '6461dda697b0a30bbd66fe36', '6461de869c50b541b2df2f1c', '6461dd1aca5873133f315e4f',
                         '6461d82f5ac5d8625a819a3a', '6461d7c08956b652879846c7', '6461d62b612b71236703b103')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [215]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230516'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id = '6464617599b931616ed29279'
"""


#Load data into pandas table
df_dispatch_copy = pd.read_sql(q, connection)

In [216]:
df_dispatch_copy

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour


In [146]:
df_dispatch2 = df_dispatch_copy2.copy()
df_dispatch2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684209910717,92980715-8bcb-7b68-910d-6545a8b97ab5,1684209910717,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646300f69ad8e2253f17829c,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":53.87,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d1580563b752c45cf8bce8e,"{""lat"":28.459147,""lng"":77.072858}",Delhi,"{""id"":""646300f69ad8e2253f17829c"",""type"":""order""}",4770.0,app,cash,"{""lat"":28.45348109269419,""lng"":77.09132214882767}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""644e0a66644efb1c5fee0bd9"",""cap...",None,None,None,None,20230516,09,0930
1,1684210168813,49842c6a-ac9d-3863-1d4b-1665bc8be8ec,1684210168813,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64630197a5c51b1fbd43d008,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":95.02,""preCommissionAmo...",5bc5ac2312477c2ece769591,62bcd17417af9a5cd065dda8,"{""lat"":28.412011773817405,""lng"":77.10839923471...",Delhi,"{""id"":""64630197a5c51b1fbd43d008"",""type"":""order""}",9400.0,app,cash,"{""lat"":28.455322801568695,""lng"":77.08740960806...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5fb55921e41162525b22df8f"",""cap...",None,None,None,None,20230516,09,0930
2,1684208237670,c5c24b54-5d38-335c-46fc-d76c6e08e6ea,1684208237670,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fa6d8e0c1952b388824b,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":30.41,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d0f9806c2a56b449fa14d5c,"{""lat"":28.4511202,""lng"":77.0965147}",Delhi,"{""id"":""6462fa6d8e0c1952b388824b"",""type"":""order""}",1770.0,app,rapido,"{""lat"":28.453746648990034,""lng"":77.0866435021162}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""636f8c0780632c4e2e6f8639"",""cap...",None,None,None,None,20230516,09,0900
3,1684208526459,44c48d21-17e7-4217-b086-39c38a899376,1684208526459,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fb8d9d1dd7505bb85d6e,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":77.12,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e27c4bb140560c235f03470,"{""lat"":28.4226457,""lng"":77.0376039}",Delhi,"{""id"":""6462fb8d9d1dd7505bb85d6e"",""type"":""order""}",7660.0,app,qrcode,"{""lat"":28.45247262182255,""lng"":77.07737680524588}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5bbc7ea84eac1c2fd102b9a1"",""cap...",None,None,None,None,20230516,09,0900
4,1684210452476,d6617278-46c5-169a-e461-bcfc4331964c,1684210452476,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646303149d1dd7505bb86f37,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":33.75,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e67934248d7c7894c00ce3e,"{""lat"":28.4538136,""lng"":77.0973243}",Delhi,"{""id"":""646303149d1dd7505bb86f37"",""type"":""order""}",1750.0,app,qrcode,"{""lat"":28.4543812986275,""lng"":77.08618953824043}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""621624f9656bb35c899caf36"",""cap...",None,None,None,None,20230516,09,0930


In [147]:
df_dispatch2['captains_with_score'] = df_dispatch2['captains_with_score'].apply(lambda x: json.loads(x))
df_dispatch2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684209910717,92980715-8bcb-7b68-910d-6545a8b97ab5,1684209910717,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646300f69ad8e2253f17829c,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":53.87,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d1580563b752c45cf8bce8e,"{""lat"":28.459147,""lng"":77.072858}",Delhi,"{""id"":""646300f69ad8e2253f17829c"",""type"":""order""}",4770.0,app,cash,"{""lat"":28.45348109269419,""lng"":77.09132214882767}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644e0a66644efb1c5fee0bd9', 'c...",None,None,None,None,20230516,09,0930
1,1684210168813,49842c6a-ac9d-3863-1d4b-1665bc8be8ec,1684210168813,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",64630197a5c51b1fbd43d008,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":95.02,""preCommissionAmo...",5bc5ac2312477c2ece769591,62bcd17417af9a5cd065dda8,"{""lat"":28.412011773817405,""lng"":77.10839923471...",Delhi,"{""id"":""64630197a5c51b1fbd43d008"",""type"":""order""}",9400.0,app,cash,"{""lat"":28.455322801568695,""lng"":77.08740960806...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5fb55921e41162525b22df8f', 'c...",None,None,None,None,20230516,09,0930
2,1684208237670,c5c24b54-5d38-335c-46fc-d76c6e08e6ea,1684208237670,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fa6d8e0c1952b388824b,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":30.41,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d0f9806c2a56b449fa14d5c,"{""lat"":28.4511202,""lng"":77.0965147}",Delhi,"{""id"":""6462fa6d8e0c1952b388824b"",""type"":""order""}",1770.0,app,rapido,"{""lat"":28.453746648990034,""lng"":77.0866435021162}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '636f8c0780632c4e2e6f8639', 'c...",None,None,None,None,20230516,09,0900
3,1684208526459,44c48d21-17e7-4217-b086-39c38a899376,1684208526459,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462fb8d9d1dd7505bb85d6e,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":77.12,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e27c4bb140560c235f03470,"{""lat"":28.4226457,""lng"":77.0376039}",Delhi,"{""id"":""6462fb8d9d1dd7505bb85d6e"",""type"":""order""}",7660.0,app,qrcode,"{""lat"":28.45247262182255,""lng"":77.07737680524588}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5bbc7ea84eac1c2fd102b9a1', 'c...",None,None,None,None,20230516,09,0900
4,1684210452476,d6617278-46c5-169a-e461-bcfc4331964c,1684210452476,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646303149d1dd7505bb86f37,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":33.75,""preCommissionAmo...",5bc5ac2312477c2ece769591,5e67934248d7c7894c00ce3e,"{""lat"":28.4538136,""lng"":77.0973243}",Delhi,"{""id"":""646303149d1dd7505bb86f37"",""type"":""order""}",1750.0,app,qrcode,"{""lat"":28.4543812986275,""lng"":77.08618953824043}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '621624f9656bb35c899caf36', 'c...",None,None,None,None,20230516,09,0930


In [45]:
df_dispatch2['mc_captain'] = df_dispatch2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9'])

In [46]:
df_filter2 = df_dispatch2[df_dispatch2['mc_captain'].astype(str) != '[]']

In [47]:
orders = df_dispatch2['order_id'].unique().tolist()
print(orders)

['6461d5a1a78efb0c7dc71e6e', '6461d60c1e1f7e2e3765351d', '6461d9b297b0a30bbd66f587', '6461d59cf4a8ae312cd1e0f8', '6461dbcfa78efb0c7dc72b91', '6461d5bd5ac5d8625a819502', '6461df4bf4a8ae312cd1f650', '6461d5fecd5d212b73a0e034', '6461dbb91e1f7e2e37654165', '6461dc088eac592e5cae996a', '6461dbc09f166a55ff604ba6', '6461d62b612b71236703b103', '6461d754780955423c322f8d', '6461db90b97fe77d4ecf91b7', '6461de08cd5d212b73a0f1cc', '6461de1df4a8ae312cd1f39c', '6461de869c50b541b2df2f1c', '6461d7141e1f7e2e37653760', '6461d9dc1e1f7e2e37653d86', '6461d82f5ac5d8625a819a3a', '6461d94f780955423c3233cd', '6461dd89780955423c323d29', '6461d8cf8956b6528798490c', '6461dedf9c50b541b2df2fc2', '6461dee4b97fe77d4ecf98c6', '6461df76751ce46ce048fbb9', '6461dee44358375373989aed', '6461dc9b612b71236703bf5b', '6461df5597b0a30bbd6701d5', '6461ddcc8eac592e5cae9d4e', '6461dd0c9f166a55ff604e47', '6461dde51e1f7e2e3765465f', '6461da69ca5873133f31585d', '6461dae5ca5873133f31596f', '6461dce9ad68a90a314dce9f', '6461dd46cd5d212b73

In [48]:
orders_not_in_suppri_logs2 = []
for order in second_captain_orders:
    if order not in orders:
        orders_not_in_suppri_logs2.append(order)

In [49]:
print(orders_not_in_suppri_logs2)

['6461dccd8753985ab62d977e', '6461dbeb8753985ab62d9584', '6461d74e751ce46ce048ea48', '6461d9938753985ab62d9080', '6461dae5cd5d212b73a0eb1e', '6461da5c9c50b541b2df2630', '6461dcf643583753739896d8', '6461d823780955423c32315d', '6461da858eac592e5cae964c', '6461d72d8753985ab62d8b4b', '6461d6ff43583753739889d9', '6461d64db97fe77d4ecf861b', '6461d6b61e1f7e2e3765368a', '6461dd2fa78efb0c7dc72e9a', '6461d8d39c50b541b2df22f7', '6461dcaf8eac592e5cae9ab3', '6461db618eac592e5cae9809', '6461de36751ce46ce048f8fe', '6461d79eb97fe77d4ecf8910', '6461de5a5ac5d8625a81a759', '6461d6f643583753739889c8', '6461dc8943583753739895fc', '6461d75d8753985ab62d8bb2', '6461d7ff612b71236703b50f', '6461d9b9780955423c3234d6', '6461dda697b0a30bbd66fe36', '6461d7c08956b652879846c7']


In [151]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
captain_list2 = df_dispatch2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['score']['micro_corridor'])

In [214]:
captain_list2 = df_dispatch2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        print(captain2['first_mile_distance'])

869.3
1004.9
1756.2
1798.8
1807.3
2026.3
2047.4
2313.5
2322.8
2405.3
2526.6
2694.1
2780.0
2859.2
3030.3
3085.5
3146.0
3156.6
3451.0
3497.1
3535.9
3591.6
3620.9
3632.8
3828.2
4425.2
4756.9
662.6
1852.2
2258.2
2315.2
2477.5
2505.6
2660.7
2946.3
2963.3
2976.1
3036.7
3200.5
3327.8
3801.8
3873.1
5409.8
328.1
682.3
721.2
1347.1
1418.7
1507.0
1813.3
1896.4
2088.5
2138.3
2147.6
2182.1
2241.2
2282.2
2368.3
2387.5
2388.1
2390.9
2410.2
2620.0
2646.8
2658.7
2779.8
2802.8
2805.3
2808.9
2819.1
2822.0
2874.0
2906.6
2912.9
2951.7
2981.9
2985.4
2997.2
3004.1
3023.4
3038.9
3080.4
3082.4
3162.0
3163.0
3180.7
3201.6
3233.8
3237.9
3260.6
3273.5
3305.9
3307.5
3315.9
3320.0
3337.8
3343.6
3381.2
3492.3
870.4
1000.7
1004.6
1058.8
1189.5
1250.6
1273.9
1282.7
1285.0
1287.6
1299.7
1308.3
1364.5
1434.2
1441.2
1454.9
1620.2
1637.6
1651.9
1689.8
1692.7
1761.6
1768.9
1778.3
1780.8
1808.9
1842.4
1847.7
1855.2
1899.9
1926.5
1939.9
1954.8
2010.9
2047.7
2055.2
2076.2
2214.5
2221.2
2266.4
2323.0
2540.6
2558.3
2596.0
2807.

In [ ]:
3404/(3404+30540)

0.1002828187603111

In [152]:
print(fm_distance2)

[2526.6, 2315.2, 2241.2, 2055.2, 2244.5, 2353.2, 2276.1, 3707.1, 2369.0, 2361.5, 2633.6, 2775.7, 2316.2, 2574.7, 2154.0, 2298.5, 2257.9, 2225.1, 2182.1, 2772.5, 2772.5, 2377.5, 3025.0, 3614.8, 2622.9, 2249.8, 2361.0, 2786.9, 2366.3, 2263.3, 2312.4, 2650.9, 2679.8, 2269.3, 3750.5, 2721.7, 2117.7, 2563.0, 2696.2, 2380.8, 2154.0, 2448.3, 2990.9, 2744.8, 2112.0, 2731.1, 2311.6, 2151.6, 2194.9, 3601.3, 2167.4, 2917.1, 2855.9, 2374.0, 2836.2, 2361.6, 2667.8, 2155.1, 3591.5, 2245.7, 2225.5, 2671.3, 2447.0, 2288.6, 2814.4, 2378.5, 2314.5, 2304.5, 3325.5, 2218.9, 2864.2, 2330.4, 3829.1, 2301.7, 2163.0, 3379.7, 2216.3, 2185.3, 2269.4, 3595.7, 3591.0, 2607.7, 3035.6, 2999.9, 2230.0, 2909.0, 2723.2, 2270.4, 2208.1, 2632.4, 2355.8, 2128.3, 2112.9, 2357.1, 2154.5, 2446.5, 2294.7, 3816.7, 2796.6, 2795.3, -1.0, 2528.7, 3519.0, 2475.8, 2348.7, 2449.7, 2418.4, 2469.0, 2273.0, 2423.3, 2293.0, 2748.2, 2548.7, 2224.1, 2449.3, 2201.8, 3780.7, 3110.4, 2359.2, 2868.4, 2303.9, 3780.6, 2380.3, 2194.3, 3412.2, 2

In [153]:
print(micro_corridor2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [65]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, event_type
from orders.order_logs_immutable
where yyyymmdd = '20230515'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
and captain_id in ('5daff8e0b2d9ba1c1cf1bdf9', '59b153ef0c3e1ca306385c51')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
"""


#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [74]:
print(len(df_apr_copy['order_id'].unique().tolist()))

93


In [67]:
#Query data from order_logs_snapshot

q = """
select * from g2n.log_dispatch_prioritiser
where yyyymmdd = '20230515'
and service_detail_id = '5da4660028af187d8d52cc3c'
and order_id in ('6461e97297b0a30bbd6717f2', '6461e8deca5873133f31786a', '6461ea1ea78efb0c7dc74aba', '6461df7c8eac592e5caea0fd', '6461e7921e1f7e2e37655b91',
 '6461e89d97b0a30bbd671622', '6461e99ef4a8ae312cd20da0', '6461e451ca5873133f316e3d', '6461ea98ca5873133f317c3f', '6461eafb612b71236703ded8',
   '6461e8101e1f7e2e37655c98', '6461df998eac592e5caea132', '6461e03c780955423c324321', '6461e72eb97fe77d4ecfab27', '6461eb705ac5d8625a81c47e',
     '6461e4f6a78efb0c7dc73f69', '64621408780955423c32acd0', '6461ee2b612b71236703e5c3', '6461e3ea9f166a55ff605cee', '6461eac29c50b541b2df4a40',
       '6461e84197b0a30bbd671561', '6461eaa7612b71236703de20', '6461e851cd5d212b73a10893', '6461e7239c50b541b2df4203', '6461e22f9c50b541b2df373b',
         '6461e171435837537398a07f', '6462138e780955423c32abc8', '6461e9158956b65287986d39', '6461e9df1e1f7e2e3765608a', '6461ece01e1f7e2e3765673f',
           '6461e55c751ce46ce04908ef', '6461ed5c97b0a30bbd672075', '6461e3429f166a55ff605b7c', '6461e5dc612b71236703d397', '6461ec62cd5d212b73a11110',
             '6461e6c58eac592e5caeb15b', '6461e9f48eac592e5caeb85c', '6461e67b1e1f7e2e37655913', '6461edeef4a8ae312cd216da', '6461e912b97fe77d4ecfaf2c',
               '6461e26f435837537398a2e1', '6461e480ca5873133f316ec6', '6462116fca5873133f31c939', '6461e357ca5873133f316bf7', '6461ef6ea78efb0c7dc7562d',
                 '6461ebca9f166a55ff606e0a', '6461e87697b0a30bbd6715ca', '6461f067ad68a90a314df871', '6461d9b9780955423c3234d6', '6461dc8f780955423c323b03',
                   '6461e8235ac5d8625a81bd4e', '6461dbeeca5873133f315bb3', '6461d9938753985ab62d9080', '6461e52d612b71236703d224', '6461db618eac592e5cae9809',
                     '6461d8d39c50b541b2df22f7', '6461da858eac592e5cae964c', '6461de36751ce46ce048f8fe', '6461d8fe8eac592e5cae933e', '6461da31b97fe77d4ecf8eac',
                       '6461dcf643583753739896d8', '6461e1ca8eac592e5caea606', '6461d760780955423c322faa', '6461dc1697b0a30bbd66fac2', '6461dcc0780955423c323b6f',
                         '6461d79eb97fe77d4ecf8910', '6461dc8943583753739895fc', '6461df4fcd5d212b73a0f49d', '6461d6b61e1f7e2e3765368a', '6461decbf4a8ae312cd1f51e',
                           '6461dcdcad68a90a314dce87', '6461d72d8753985ab62d8b4b', '6461d64db97fe77d4ecf861b', '6461dd2fa78efb0c7dc72e9a', '6461d823780955423c32315d',
                             '6461dd2cca5873133f315e73', '6461d82f5ac5d8625a819a3a', '6461dc4f612b71236703beb4', '6461dae5cd5d212b73a0eb1e',
                               '6461d9385ac5d8625a819c48', '6461dda697b0a30bbd66fe36', '6461d75d8753985ab62d8bb2', '6461db9343583753739893df',
                                 '6461d6ff43583753739889d9', '6461de5a5ac5d8625a81a759', '646246ce3c976113e233feb0', '6461de3f435837537398999e',
                                   '6461dbeb8753985ab62d9584', '6461dccd8753985ab62d977e', '6461ddfccd5d212b73a0f1b1', '6461deed751ce46ce048fa74',
                                     '6461d969b97fe77d4ecf8d01', '6461df2c8753985ab62d9c8f')
"""


#Load data into pandas table
df_dispatch_copy2 = pd.read_sql(q, connection)

In [68]:
df_dispatch_copy2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684140292418,7135eb5c-0a29-e9de-26b8-be863d5ffb81,1684140292418,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":31.93,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""63b2d840cb0cad3e7cbac392"",""cap...",None,None,None,None,20230515,14,1400
1,1684139563474,05253f4d-8209-f8e5-3fd8-b2953abf828e,1684139563474,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ee2b612b71236703e5c3,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":37.01,""preCommissionAmo...",5bc5ac2312477c2ece769591,618c9773c98cb31c05fb2266,"{""lat"":28.474934,""lng"":77.084606}",Delhi,"{""id"":""6461ee2b612b71236703e5c3"",""type"":""order""}",3250.0,app,paytm,"{""lat"":28.454235975103476,""lng"":77.08387613296...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5daff8e0b2d9ba1c1cf1bdf9"",""cap...",None,None,None,None,20230515,14,1400
2,1684135834295,d0311a16-ef5c-4f66-c522-52cd284ac53b,1684135834295,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461df998eac592e5caea132,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":27.57,""preCommissionAmo...",5bc5ac2312477c2ece769591,637b581ffbd64e460e8f14f1,"{""lat"":28.4524237,""lng"":77.069357}",Delhi,"{""id"":""6461df998eac592e5caea132"",""type"":""order""}",2720.0,app,paytm,"{""lat"":28.46185325257259,""lng"":77.08687350153923}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""5d10ac1a0eed7c44be2c1fb5"",""cap...",None,None,None,None,20230515,13,1300
3,1684139945562,5bd9cdb5-8473-671e-2b04-9f2c3b933453,1684139945562,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":42.61,""preCommissionAmo...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""635cc2bed58af76104dc3473"",""cap...",None,None,None,None,20230515,14,1400
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{""captain_id"":""63aa660aff12f9b46e7e6c6f"",""cap...",None,None,None,None,20230515,14,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,1684136939711,65d3f

In [75]:
df_dispatch_copy2['order_id'].nunique()

59

In [78]:
df_dispatch_copy2['captains_with_score'] = df_dispatch_copy2['captains_with_score'].apply(lambda x: json.loads(x))
df_dispatch_copy2.head()

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour
0,1684140292418,7135eb5c-0a29-e9de-26b8-be863d5ffb81,1684140292418,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":31.93,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400
1,1684139563474,05253f4d-8209-f8e5-3fd8-b2953abf828e,1684139563474,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ee2b612b71236703e5c3,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""earnings"":{""amount"":37.01,""preCommissionAmo...",5bc5ac2312477c2ece769591,618c9773c98cb31c05fb2266,"{""lat"":28.474934,""lng"":77.084606}",Delhi,"{""id"":""6461ee2b612b71236703e5c3"",""type"":""order""}",3250.0,app,paytm,"{""lat"":28.454235975103476,""lng"":77.08387613296...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5daff8e0b2d9ba1c1cf1bdf9', 'c...",None,None,None,None,20230515,14,1400
2,1684135834295,d0311a16-ef5c-4f66-c522-52cd284ac53b,1684135834295,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461df998eac592e5caea132,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":27.57,""preCommissionAmo...",5bc5ac2312477c2ece769591,637b581ffbd64e460e8f14f1,"{""lat"":28.4524237,""lng"":77.069357}",Delhi,"{""id"":""6461df998eac592e5caea132"",""type"":""order""}",2720.0,app,paytm,"{""lat"":28.46185325257259,""lng"":77.08687350153923}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5d10ac1a0eed7c44be2c1fb5', 'c...",None,None,None,None,20230515,13,1300
3,1684139945562,5bd9cdb5-8473-671e-2b04-9f2c3b933453,1684139945562,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":42.61,""preCommissionAmo...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '635cc2bed58af76104dc3473', 'c...",None,None,None,None,20230515,14,1400
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400


In [82]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
fm_distance = []
captain_rank = []
micro_corridor = []
captain_list2 = df_dispatch_copy2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['micro_corridor_enabled'])
        elif captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance.append(captain2['first_mile_distance'])
            captain_rank.append(captains2.index(captain2))
            micro_corridor.append(captain2['micro_corridor_enabled'])

In [187]:
for x in df_dispatch_copy2['captains_with_score'].values[0]:
    if x['captain_id'] == '59b153ef0c3e1ca306385c51':
        print(x['location'])

{'lat': 28.438816692504904, 'lng': 77.07940354943275}


In [189]:
h3.geo_to_h3(28.438816692504904, 77.07940354943275, resolution=8)

'883da11ab5fffff'

In [83]:
print(fm_distance2)

[407.9, 1117.0, 1112.4, 498.9, 347.0, 298.7, 3443.0, 396.3, 1777.6, 2344.0, 1795.7, 3429.6, 345.2, 328.9, 1136.3, 3388.0, 3737.8, 47.4, 735.0, 3519.1, 2793.8, 2793.0, 1161.0, 1704.4, 3443.1, 1796.4, 3395.4, 3737.3, 3750.8, 3441.7, 1152.3, 1819.5, 3395.4, 730.8, 88.2, 2366.0, 1133.5, 2793.9, 3381.0, 1104.6, 914.2, 1938.3, 3518.2, 1953.2, 1792.9, 27.2, 1191.7, 1787.5, 3434.9, 2244.0, 340.8, 2194.8, 1090.6, 1203.6, 1777.6, 90.2, 2369.2, 2744.0, 1793.4, 920.4, 2391.1, 1936.3, 2127.0, 1679.7, 1931.3, 1775.8, 178.8, 102.6, 25.7, 2544.4, 2786.8, 1159.7, 2195.1, 44.5, 3531.0, 2585.0, 2550.8, 1580.9, 1693.8, 3743.4, 2247.2, 2751.4, 1795.9]


In [84]:
print(fm_distance)

[938.4, 938.4, 2290.0, 684.1, 496.1, 935.8, 337.1, 24.0, 2463.7]


In [85]:
print(captain_rank)

[4, 3, 1, 1, 1, 0, 2, 0, 52]


In [87]:
df_dispatch_copy2['mc_captain_1'] = df_dispatch_copy2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9'])
df_dispatch_copy2['mc_captain_2'] = df_dispatch_copy2['captains_with_score'].apply(lambda x:
                                                                     [captain['captain_id'] for captain in x if captain['captain_id'] == '59b153ef0c3e1ca306385c51'])

In [120]:
df_filter2 = df_dispatch_copy2[df_dispatch_copy2['mc_captain_2'].astype(str) != '[]']
df_filter2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour,mc_captain_1,mc_captain_2
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
5,1684140171298,de01dc8a-ba2f-d9f3-bb80-8291681fde86,1684140171298,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '62863d9e8f9b4309675e2665', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
14,1684140068293,026fe352-e279-6348-1d6f-c26c06ed1a48,1684140068293,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""firstMileDistance"":3318.7,""id"":""612085be01c...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
18,1684149134719,b1018bd7-e22e-421f-a9eb-9f61d5cb2e9e,1684149134719,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462138e780955423c32abc8,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":47.16,""preCommissionAmo...",5bc5ac2312477c2ece769591,600f2a5a027f256eb8a33e1e,"{""lat"":28.4312655684068,""lng"":77.0443594828248}",Delhi,"{""id"":""6462138e780955423c32abc8"",""type"":""order""}",4330.0,app,simpl,"{""lat"":28.459418566032934,""lng"":77.04956263303...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5de8ad2523da8b6b4b802cf5', 'c...",None,None,None,None,20230515,16,1630,[],[59b153ef0c3e1ca306385c51]
19,1684162255544,b298d76b-071f-a2c1-1053-34c6b0ac0ba8,1684162255544,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646246ce3c976113e233feb0,,dispatch,"{""dropped_to_total_pings_ratio"":{""bucket"":{""pa...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":20.0,""preCommissionAmou...",5bc5ac2312477c2ece769591,61c811d45379c0d4d24f1e92,"{""lat"":28.4392927,""lng"":77.0797819}",Delhi,"{""id"":""646246ce3c976113e233feb0"",""type"":""order""}",540.0,app,paytm,"{""lat"":28.4411481,""lng"":77.07658931}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644f68a2e80c7dfa1ead1ce9', 'c...",None,None,None,None,20230515,20,2015,[],[59b153ef0c3e1ca306385c51

In [111]:
df_filter2['enabled_strategies_with_config'].values[0]

'{"first_mile_distance":{"bucket":{"params":[{"primaryBucketEnd":1000,"primaryBucketStart":0,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100},{"primaryBucketEnd":2000,"primaryBucketStart":1000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100},{"primaryBucketEnd":10000,"primaryBucketStart":2000,"secondaryBucketEnd":100000,"secondaryBucketStart":0,"weightage":100}],"type":"first_mile_distance"}}}'

In [92]:
fm_distance2 = []
captain_rank2 = []
micro_corridor2 = []
captain_list2 = df_filter2['captains_with_score'].values
for captains2 in captain_list2:
    for captain2 in captains2:
        if captain2['captain_id'] == '59b153ef0c3e1ca306385c51':
            fm_distance2.append(captain2['first_mile_distance'])
            captain_rank2.append(captains2.index(captain2))
            micro_corridor2.append(captain2['micro_corridor_enabled'])

In [93]:
print(captain_rank)

[4, 3, 1, 1, 1, 0, 2, 0, 52]


In [94]:
print(fm_distance)

[938.4, 938.4, 2290.0, 684.1, 496.1, 935.8, 337.1, 24.0, 2463.7]


In [121]:
df_filter2

,created_on,event_id,updated_at,version,enabled_captain_filter_types,order_id,compositeHexId,prioritization_stage,enabled_strategies_with_config,filtered_captains,captains,city_id,customer_id,drop_location,city_name,correlation,last_mile_distance,order_type,payment_type,pickup_location,service_category,service_detail_id,retry_count,captains_with_score,enabled_strategies_with_scores,epkm,infeasible_captains,prioritized_captains,yyyymmdd,hh,quarter_hour,mc_captain_1,mc_captain_2
4,1684140136342,20f41813-4c08-6af6-e658-6b0781c35555,1684140136342,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63aa660aff12f9b46e7e6c6f', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
5,1684140171298,de01dc8a-ba2f-d9f3-bb80-8291681fde86,1684140171298,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461f067ad68a90a314df871,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":24.76,""preCommissionAmo...",5bc5ac2312477c2ece769591,5d8e0c04a1f0a71c604b69f5,"{""lat"":28.4527467,""lng"":77.0510669}",Delhi,"{""id"":""6461f067ad68a90a314df871"",""type"":""order""}",1090.0,app,cash,"{""lat"":28.4542353855553,""lng"":77.05701012164354}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '62863d9e8f9b4309675e2665', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
14,1684140068293,026fe352-e279-6348-1d6f-c26c06ed1a48,1684140068293,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6461ef6ea78efb0c7dc7562d,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...",[],"[{""firstMileDistance"":3318.7,""id"":""612085be01c...",5bc5ac2312477c2ece769591,640326b98e212c0c234dd93a,"{""lat"":28.4542434,""lng"":77.0851923}",Delhi,"{""id"":""6461ef6ea78efb0c7dc7562d"",""type"":""order""}",2810.0,app,cash,"{""lat"":28.458548709966106,""lng"":77.07276653274...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '63b2d840cb0cad3e7cbac392', 'c...",None,None,None,None,20230515,14,1400,[],[59b153ef0c3e1ca306385c51]
18,1684149134719,b1018bd7-e22e-421f-a9eb-9f61d5cb2e9e,1684149134719,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",6462138e780955423c32abc8,,dispatch,"{""first_mile_distance"":{""bucket"":{""params"":[{""...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":47.16,""preCommissionAmo...",5bc5ac2312477c2ece769591,600f2a5a027f256eb8a33e1e,"{""lat"":28.4312655684068,""lng"":77.0443594828248}",Delhi,"{""id"":""6462138e780955423c32abc8"",""type"":""order""}",4330.0,app,simpl,"{""lat"":28.459418566032934,""lng"":77.04956263303...",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '5de8ad2523da8b6b4b802cf5', 'c...",None,None,None,None,20230515,16,1630,[],[59b153ef0c3e1ca306385c51]
19,1684162255544,b298d76b-071f-a2c1-1053-34c6b0ac0ba8,1684162255544,1.3.0,"[""economicFeasibilityOfARide"",""failedMatchInTh...",646246ce3c976113e233feb0,,dispatch,"{""dropped_to_total_pings_ratio"":{""bucket"":{""pa...","[{""filter"":[{""meta"":{""first_mile_percent_slack...","[{""earnings"":{""amount"":20.0,""preCommissionAmou...",5bc5ac2312477c2ece769591,61c811d45379c0d4d24f1e92,"{""lat"":28.4392927,""lng"":77.0797819}",Delhi,"{""id"":""646246ce3c976113e233feb0"",""type"":""order""}",540.0,app,paytm,"{""lat"":28.4411481,""lng"":77.07658931}",app,5da4660028af187d8d52cc3c,0,"[{'captain_id': '644f68a2e80c7dfa1ead1ce9', 'c...",None,None,None,None,20230515,20,2015,[],[59b153ef0c3e1ca306385c51

In [114]:
captains_logs = df_filter2['captains_with_score'].values
for log in captains_logs:
    for individual_log in log:
        if individual_log['captain_id'] == '5daff8e0b2d9ba1c1cf1bdf9':
            print('MC Score', individual_log['score']['micro_corridor'])
            print('FM', individual_log['first_mile_distance'])
            print(individual_log['micro_corridor_enabled'])

MC Score 0
FM 407.9
True
MC Score 0
FM 1117.0
True
MC Score 0
FM 1112.4
True
MC Score 0
FM 498.9
True
MC Score 0
FM 347.0
True
MC Score 0
FM 298.7
True
MC Score 0
FM 3443.0
True
MC Score 0
FM 396.3
True
MC Score 0
FM 1777.6
True
MC Score 0
FM 2344.0
True
MC Score 0
FM 1795.7
True
MC Score 0
FM 3429.6
True
MC Score 0
FM 345.2
True
MC Score 0
FM 328.9
True
MC Score 0
FM 1136.3
True
MC Score 0
FM 3388.0
True
MC Score 0
FM 3737.8
True
MC Score 0
FM 47.4
True
MC Score 0
FM 735.0
True
MC Score 0
FM 3519.1
True
MC Score 0
FM 2793.8
True
MC Score 0
FM 2793.0
True
MC Score 0
FM 1161.0
True
MC Score 0
FM 1704.4
True
MC Score 0
FM 3443.1
True
MC Score 0
FM 1796.4
True
MC Score 0
FM 3395.4
True
MC Score 0
FM 3737.3
True
MC Score 0
FM 3750.8
True
MC Score 0
FM 3441.7
True
MC Score 0
FM 1152.3
True
MC Score 0
FM 1819.5
True
MC Score 0
FM 3395.4
True
MC Score 0
FM 730.8
True
MC Score 0
FM 88.2
True
MC Score 0
FM 2366.0
True
MC Score 0
FM 1133.5
True
MC Score 0
FM 2793.9
True
MC Score 0
FM 3381.0
True

In [106]:
captains_logs[1][0]

{'captain_id': '62863d9e8f9b4309675e2665',
 'captain_segment': 0,
 'drop_location_score': 0,
 'dropped_to_total_pings_ratio': 0.0,
 'earnings_per_hour': 0,
 'epkm': 14.549728,
 'eta': 0,
 'first_mile_distance': 782.2,
 'handholding': False,
 'high_last_mile_distance_affinity': 0,
 'is_eligible_for_deprioritization': False,
 'is_preferred_route': False,
 'location': {'lat': 28.448918116883434, 'lng': 77.05793883651495},
 'micro_corridor_enabled': False,
 'probability_of_accept': 0,
 'route_preference': {'enabled': False, 'location': {}},
 'score': {'captain_segment': 0.0,
  'deprioritization': 0,
  'drop_location_preference': 0,
  'dropped_to_total_pings_ratio': 0.0,
  'earnings_per_hour': 0,
  'eta': 0,
  'first_mile_distance': 0.0,
  'handholding': 0,
  'high_last_mile_distance_affinity': 0,
  'micro_corridor': 0,
  'probability_of_accept': 0,
  'route_preference': 0,
  'service_affinity': 0},
 'service_affinity': 0,
 'total_score': 0.0}